In [1]:
import pandas as pd
import akshare as ak

etfs = ['561300', '159726', '515100', '513500', '161119', '518880', '164824', '159985', '513330', '513100', '513030', '513520']

df = pd.DataFrame()

end_date = '20250731'

for i in etfs:
    etf_df = ak.fund_etf_hist_em(symbol=i, period="daily", start_date="20220701", end_date=end_date, adjust="hfq")
    etf_df['close'] = etf_df['收盘'].astype(float)
    etf_df['date'] = etf_df['日期']
    etf_df['symbol'] = i
    etf_df = etf_df.ffill()
    etf_df = etf_df.infer_objects(copy=False)
    df = pd.concat([df, etf_df])

/home/jovyan/data/.venv/lib/python3.11/site-packages/py_mini_racer/py_mini_racer.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


  0%|          | 0/12 [00:00<?, ?it/s]

In [1]:
import pandas as pd
import duckdb

# 要查询的 ETF 列表
etfs = ['561300', '159726', '515100', '513500', '161119', '518880', '164824', '159985', '513330', '513100', '513030', '513520']

# 查询的起止日期
start_date = '2022-07-01'
end_date = '2025-08-13'

# 连接到 DuckDB 数据库 (只读模式)
path = '../dataset/data/etf_prices/**/data.parquet'
con = duckdb.connect(database=':memory:', read_only=False) 

# 构建 SQL 查询语句
# 将 etfs 列表转换为适合 SQL "IN" 子句的元组格式
symbols_tuple = tuple(etfs)

query = f"""
SELECT
    date,
    close,
    symbol
FROM read_parquet('{path}')
WHERE
    symbol IN {symbols_tuple}
    AND date BETWEEN '{start_date}' AND '{end_date}'
ORDER BY
    symbol, date
"""

# 执行查询并将结果加载到 pandas DataFrame
try:
    df = con.execute(query).fetchdf()
finally:
    # 关闭数据库连接
    con.close()

# 将 'close' 列转换为浮点数类型
df['close'] = df['close'].astype(float)

# 将 'date' 列转换为日期时间类型
df['date'] = pd.to_datetime(df['date'])

# 对每个 symbol 分组后进行前向填充 (ffill)
# 这是为了匹配您原始脚本中的逻辑
df = df.groupby('symbol', group_keys=False).apply(lambda x: x.ffill())

# 重新推断数据类型，以匹配原始脚本
df = df.infer_objects(copy=False)

# 打印 DataFrame 的前几行以供查阅
print(df.head())

        date  close  symbol
0 2022-07-01  1.007  159726
1 2022-07-04  1.008  159726
2 2022-07-05  1.006  159726
3 2022-07-06  0.992  159726
4 2022-07-07  0.997  159726


/tmp/ipykernel_16594/1035125006.py:47: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('symbol', group_keys=False).apply(lambda x: x.ffill())


In [2]:
df_pivot = df.pivot(index='date', columns='symbol', values='close')
df_pivot = df_pivot.dropna()
df_pivot

symbol,159726,159985,161119,164824,513030,513100,513330,513500,513520,515100,518880,561300
date,,,,,,,,,,,,
2022-07-01,1.007,1.527,1.544,1.091,0.910,3.995,0.524,2.382,0.970,1.540,3.784,0.866
2022-07-04,1.008,1.510,1.544,1.093,0.912,4.010,0.533,2.412,0.972,1.541,3.801,0.872
2022-07-05,1.006,1.514,1.543,1.103,0.910,4.070,0.533,2.432,0.973,1.537,3.797,0.870
2022-07-06,0.992,1.499,1.543,1.095,0.874,4.095,0.520,2.420,0.969,1.518,3.725,0.858
2022-07-07,0.997,1.531,1.543,1.109,0.878,4.145,0.521,2.442,0.977,1.520,3.676,0.864
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-07,1.298,1.975,1.762,1.471,1.867,8.420,0.532,4.348,1.547,1.978,7.484,0.882
2025-08-08,1.307,1.984,1.762,1.474,1.881,8.455,0.524,4.348,1.574,1.981,7.508,0.882
2025-08-11,1.300,1.969,1.761,1.474,1.889,8.540,0.526,4.384,1.590,1.979,7.429,0.884


In [3]:
from sklearn import set_config
from sklearn.model_selection import (
    GridSearchCV,
    KFold,
    RandomizedSearchCV,
    train_test_split
)
from sklearn.pipeline import Pipeline
from scipy.stats import loguniform
import matplotlib.pyplot as plt

from skfolio import RatioMeasure, RiskMeasure
from skfolio.datasets import load_factors_dataset, load_sp500_dataset
from skfolio.distance import KendallDistance
from skfolio.model_selection import (
    CombinatorialPurgedCV,
    WalkForward,
    cross_val_predict,
)
from skfolio.moments import (
    DenoiseCovariance,
    DetoneCovariance,
    EWMu,
    GerberCovariance,
    ShrunkMu,
    ShrunkCovariance
)
from skfolio.optimization import (
    MeanRisk,
    NestedClustersOptimization,
    ObjectiveFunction,
    RiskBudgeting,
    HierarchicalRiskParity,
    DistributionallyRobustCVaR,
    StackingOptimization,
    MaximumDiversification,
    HierarchicalEqualRiskContribution,
    InverseVolatility,
    ConvexOptimization
)
from skfolio.pre_selection import SelectKExtremes
from skfolio.preprocessing import prices_to_returns
from skfolio.prior import BlackLitterman, EmpiricalPrior, FactorModel
from skfolio.uncertainty_set import BootstrapMuUncertaintySet
from skfolio.portfolio import MultiPeriodPortfolio
from skfolio.cluster import HierarchicalClustering, LinkageMethod

In [4]:
prices = df_pivot.copy()
prices = prices.sort_values(by='date').dropna()

cv = WalkForward(test_size=1, train_size=7, freq="MS")


model = HierarchicalRiskParity(
    risk_measure=RiskMeasure.CVAR,
    prior_estimator=EmpiricalPrior(
        covariance_estimator=ShrunkCovariance(shrinkage=0.9)
    )
)

model3 = RiskBudgeting(
    risk_measure=RiskMeasure.EVAR,
    portfolio_params=dict(name="Risk Parity - Covariance Shrinkage")
    # solver='SCS'
)

estimators = [
    ("model1", model),
    ("model2", MaximumDiversification()),
    ("model3", DistributionallyRobustCVaR(wasserstein_ball_radius=0.01))
]

model_stacking = StackingOptimization(
    estimators=estimators,
    final_estimator=model3,
)

X = prices_to_returns(prices)
X.index = pd.to_datetime(X.index)

pred_stacking = cross_val_predict(
    model_stacking,
    X,
    cv=cv,
    n_jobs=-1,
    portfolio_params=dict(name="Stacking")
)

In [5]:
returns = pred_stacking.plot_cumulative_returns()
returns.show(renderer='iframe')

In [13]:
cumulative_returns = pred_stacking.returns

0      0.000000
1      0.000000
2      0.000000
3      0.000000
4      0.000000
         ...   
753    0.002873
754    0.001430
755    0.004228
756    0.000486
757    0.007041
Name: return, Length: 758, dtype: float64

In [11]:
returns

,index,return
0,2022-07-01 00:00:00+00:00,0.000000
1,2022-07-04 00:00:00+00:00,0.000000
2,2022-07-05 00:00:00+00:00,0.000000
3,2022-07-06 00:00:00+00:00,0.000000
4,2022-07-07 00:00:00+00:00,0.000000
...,...,...
753,2025-08-07 00:00:00+00:00,0.002873
754,2025-08-08 00:00:00+00:00,0.001430
755,2025-08-11 00:00:00+00:00,0.004228
756,2025-08-12 00:00:00+00:00,0.000486


In [20]:
composition = pred_stacking.plot_composition()
composition.show(renderer='iframe')

In [21]:
pred_stacking.summary()

Mean                                     0.040%
Annualized Mean                          10.12%
Variance                               0.00077%
Annualized Variance                       0.19%
Semi-Variance                          0.00044%
Annualized Semi-Variance                  0.11%
Standard Deviation                        0.28%
Annualized Standard Deviation             4.40%
Semi-Deviation                            0.21%
Annualized Semi-Deviation                 3.32%
Mean Absolute Deviation                   0.19%
CVaR at 95%                               0.60%
EVaR at 95%                               1.53%
Worst Realization                         2.59%
CDaR at 95%                               2.13%
MAX Drawdown                              3.11%
Average Drawdown                          0.48%
EDaR at 95%                               2.37%
First Lower Partial Moment               0.096%
Ulcer Index                              0.0074
Gini Mean Difference                    

In [19]:
pred_stacking.weights_per_observation

,159726,159985,161119,164824,513030,513100,513330,513500,513520,515100,518880,561300
2022-09-01,0.018662,0.028534,0.670745,0.053054,0.021103,0.021200,0.013135,0.017874,0.019541,0.017503,0.101688,0.016961
2022-09-02,0.018662,0.028534,0.670745,0.053054,0.021103,0.021200,0.013135,0.017874,0.019541,0.017503,0.101688,0.016961
2022-09-05,0.018662,0.028534,0.670745,0.053054,0.021103,0.021200,0.013135,0.017874,0.019541,0.017503,0.101688,0.016961
2022-09-06,0.018662,0.028534,0.670745,0.053054,0.021103,0.021200,0.013135,0.017874,0.019541,0.017503,0.101688,0.016961
2022-09-07,0.018662,0.028534,0.670745,0.053054,0.021103,0.021200,0.013135,0.017874,0.019541,0.017503,0.101688,0.016961
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-24,0.016205,0.058234,0.676137,0.030993,0.030080,0.009944,0.027933,0.016175,0.022993,0.042024,0.045668,0.023614
2025-06-25,0.016205,0.058234,0.676137,0.030993,0.030080,0.009944,0.027933,0.016175,0.022993,0.042024,0.045668,0.023614
2025-06-26,0.016205,0.058234,0.676137,0.030993,0.030080,0.009944,0.027933,0.016175,0.022993,0.042024,0.045668,0.023614
2025-06-27,0.016205,0.058234,0.676137,0.030993,0.030080,0.009944,0.027933,0.016175,0.022993,0.042024,0.045668,0.023614


In [20]:
df_c = df_pivot.copy()
df_c = prices_to_returns(df_c)
df_c.index = pd.to_datetime(df_c.index)

end_date = df_c.index.max()

train_start_date = (end_date - pd.DateOffset(months=7))
train_end_date = (end_date - pd.DateOffset(months=1))

train_df = df_c.loc[train_start_date:train_end_date]
predict_df = df_c.loc[train_end_date:]

model_stacking.fit(train_df)

pred_weight = model_stacking.predict(predict_df)

In [21]:
train_start_date

Timestamp('2024-12-31 00:00:00')

In [22]:
weight_df = pred_weight.weights_per_observation.tail(6)
weight_df.round(2)

,159726,159985,161119,164824,513030,513100,513330,513500,513520,515100,518880,561300
2025-07-24,0.02,0.06,0.67,0.03,0.03,0.01,0.03,0.02,0.02,0.04,0.04,0.03
2025-07-25,0.02,0.06,0.67,0.03,0.03,0.01,0.03,0.02,0.02,0.04,0.04,0.03
2025-07-28,0.02,0.06,0.67,0.03,0.03,0.01,0.03,0.02,0.02,0.04,0.04,0.03
2025-07-29,0.02,0.06,0.67,0.03,0.03,0.01,0.03,0.02,0.02,0.04,0.04,0.03
2025-07-30,0.02,0.06,0.67,0.03,0.03,0.01,0.03,0.02,0.02,0.04,0.04,0.03
2025-07-31,0.02,0.06,0.67,0.03,0.03,0.01,0.03,0.02,0.02,0.04,0.04,0.03
